## В рамках данной лабы буду проводить кластризацию текста русских шуток. Целевой переменной будет тема шутки, посмотрим, каким образом алгоритм будет кластеризовать текста и будет ли это связано с темой шутки

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud
import seaborn as sns

In [10]:
plt.rcParams["figure.figsize"] = (20,10)

In [11]:
data = pd.read_csv("../input/russian-jokes/jokes.csv")

In [12]:
data.head()

In [13]:
data.info()

У нас нет пропущенных значений, отлично!

Посмотрим количество уникальных значений

In [14]:
data.nunique()

Посмотрим распределение шуток по темам

In [15]:
plt.xticks(rotation = 90)
sns.countplot(x = 'theme', data = data)

In [16]:
data['theme'].value_counts()

Видим неравномерное распределение. Чтобы хоть как-то можно было описать полученный результат и увидеть на графиках уменьшим вариативность и оставим только 8 классов, которые между собой распределены +- равномерно

- pro-studentov
- pro-militsiyu
- pro-armiu
- poshlie-i-intimnie
- pro-detey
- meditsinskie
- pro-mugchin
- shkolnie-i-pro-shkolu

In [17]:
data = data[data['theme'].isin(['pro-studentov', 'pro-militsiyu', 'pro-armiu', 'poshlie-i-intimnie', 'pro-detey', 'meditsinskie', 'pro-mugchin', 'shkolnie-i-pro-shkolu'])]

In [18]:
plt.xticks(rotation = 90)
sns.countplot(x = 'theme', data = data)

In [19]:
data

Проведем базовый препроцессинг (очистим от знаков препинания и приведем к нижнему регистру)

In [20]:
import re
def clean(text):
    text = re.sub('[^А-Яа-я]+', ' ', text)
    return text

def lower(text):
    return text.lower()

data['textPrep1'] = data['text'].apply(clean)
data['textPrep1'] = data['textPrep1'].apply(lower)
data.head()

Закодируем наши темы с помощью LabelEncoder ( в будущем будем строить графики где лейблы будут цветами )

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [22]:
data['theme']

In [23]:
data['themeEncoded'] = le.fit_transform(data['theme'])

Для получения эмбеддингов будем использовать TfIdfVectorizer

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorizer = TfidfVectorizer()
x_prep1 = vectorizer.fit_transform(data['textPrep1'])

In [26]:
vectorizer.get_feature_names()[:30]

видим что из-за того что мы не сделали лемматизацию слова абзац и абзаца считаются разными

Изобразим 3d график распределения тем

Чтобы нам его отрисовать нужно уменьшить размерность наших векторов до 3мерного пространства. Используем TruncatedSVD так как он отлично работает со sparse матрицами (которую производит наш tfIdfVectorizer например) и не съедает тонну памяти как PCA

In [27]:
from sklearn.decomposition import TruncatedSVD

In [28]:
x_prep1_svd = TruncatedSVD(3).fit_transform(x_prep1.toarray())

In [29]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_prep1_svd[:,0], x_prep1_svd[:,1],x_prep1_svd[:,2], c = data['themeEncoded'])

Данные очень скученны и сложно визуально их отделить друг от друга. Вероятно, если различие и есть, то на больших размерностях

Для кластеризации будем использовать алгоритм kmeans с числом классов = числу наших меток (8)

In [30]:
from sklearn.cluster import KMeans

In [31]:
kmeans = KMeans(n_clusters = 8, n_init = 5, random_state = 228)
kmeans.fit(x_prep1)

Визуализируем получившиеся метки и то как они совпадают с нашими темами с помощью таблицы

In [32]:
data['clusterPrep1'] = kmeans.labels_

In [33]:
clusters = data.groupby(['clusterPrep1', 'theme']).size()

In [34]:
sns.heatmap(clusters.unstack(level = 'theme'), annot = True, fmt='g')

In [35]:
data['clusterPrep1'].value_counts()

In [36]:
sns.countplot(x = data['clusterPrep1'])

Видим, что данные разделились по кластерам весьма неравномерно, во второй и 6 кластера попало больше всего данных. Также данные размазаны по всем кластерам, точность неудовлетворительная

Интересно, что алгоритм в 5 кластер определил большинство анекдотов "про детей"

Снова построим 3д график меток

In [37]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_prep1_svd[:,0], x_prep1_svd[:,1],x_prep1_svd[:,2], c = data['clusterPrep1'])

Теперь проведем более продвинутый препроцессинг. Возможно это улучшит результат

Токенизируем получившийся текст (Разбиваем предложение на токены (в нашем случае - слова))

In [38]:
from nltk.tokenize import word_tokenize

In [39]:
data['tokens'] = data['textPrep1'].apply(word_tokenize)
data.head()

Удалим токены с длиной < 3 и > 20

In [40]:
def delete_spam(text):
    result = []
    for word in text:
        if len(word) >= 3 and len(word) <= 20:
            result.append(word)
    return result

In [41]:
data['tokens'] = data['tokens'].apply(delete_spam)

Удалим стоп слова (общеупотребительные слова, не дающие полезной нагрузки для анализа)

In [42]:
import nltk
from nltk.corpus import stopwords

In [43]:
stopword_set = set(stopwords.words('russian'))

def token_remove_stop(tokens):
    res = []
    for word in tokens:
        if word not in stopword_set:
            res.append(word)
    return res

In [44]:
data['tokens'] = data['tokens'].apply(token_remove_stop)

Выполним лемматизацию (приведение слов к начальной форме)

In [47]:
import spacy

In [48]:
!python -m spacy download ru_core_news_md

In [60]:
nlp = spacy.load("ru_core_news_sm", disable=['parser', 'tagger', 'ner'])

In [61]:
def lemmatize(tokens):
    sentence = ' '.join(tokens)
    return [token.lemma_ for token in nlp(sentence)]

In [62]:
data['tokens'] = data['tokens'].apply(lemmatize)

In [63]:
data['textPrep2'] = data['tokens'].apply(lambda tokens: ' '.join(tokens))

Попробуем снова обучить нашу kmeans модель и проделать все те же шаги что делали до этого

In [64]:
vectorizer = TfidfVectorizer()
x_prep2 = vectorizer.fit_transform(data['textPrep2'])

In [65]:
x_prep2_svd = TruncatedSVD(3).fit_transform(x_prep2.toarray())

In [66]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_prep2_svd[:,0], x_prep2_svd[:,1],x_prep2_svd[:,2], c = data['themeEncoded'])

График функции сильно поменялся, но сказать что-либо все равно сложно, так как кластеры выделены неявным образом

In [67]:
kmeans = KMeans(n_clusters = 8, n_init = 5, random_state = 228)
kmeans.fit(x_prep2)

In [68]:
data['clusterPrep2'] = kmeans.labels_

In [69]:
clusters = data.groupby(['clusterPrep2', 'theme']).size()
sns.heatmap(clusters.unstack(level = 'theme'), annot = True, fmt='g')

In [70]:
data['clusterPrep2'].value_counts()

In [71]:
sns.countplot(x = data['clusterPrep2'])

Снова видим доминирующий кластер, теперь это кластер с номером 0. Остальные кластеры стали еще меньше, ОДНАКО, если не брать во внимание кластер 0 (куда, возможно, попали анекдоты, которым невозможно определить тему) то в остальных кластерах преобладает по одной конкретной теме (pro-armiu в 4, pro-detey в 5, pro-studentov во 2, meditsinskie в 1)

In [73]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_prep2_svd[:,0], x_prep2_svd[:,1],x_prep2_svd[:,2], c = data['clusterPrep2'])

Все также ничего не понятно :( 

## Выводы: 

Обе модели разделили на кластера не лучшим образом, вероятно, из-за пересечения разных тем. Однако в отличие от первой модели, кластера которой включали сразу много тем анекдотов, вторая модель разделила анекдоты по кластерам наиболее приближенно к разделению по темам.